<a href="https://colab.research.google.com/github/TryDovick/MosiNLP/blob/main/%D0%94%D0%BE%D0%B1%D1%80%D0%BE_%D0%BF%D0%BE%D0%B6%D0%B0%D0%BB%D0%BE%D0%B2%D0%B0%D1%82%D1%8C_%D0%B2_Colab!.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Классические методы преобразования текста

Установка библиотек

In [ ]:
!pip install pymorphy3

In [ ]:
import pymorphy3
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import math
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
from math import log

Установка ресурсов nltk

In [ ]:
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Текст ввода

In [ ]:
text = ["Хеш-таблица хранит данные в виде ключ-значение",
        "Полиморфизм позволяет использовать один интерфейс для разных типов",
        "Рекурсия — это вызов функцией самой себя"
        ]

Лемантизация и токенизация

In [ ]:
def preprocessing(text: list[str]) -> list[list[str]]:
    stemmer = SnowballStemmer("russian")  # Используем стеммер для русского языка
    stop_words = set(stopwords.words("russian"))  # Русские стоп-слова
    preprocessed_texts = []

    for text in text:
        tokens = word_tokenize(text.lower(), language='russian')  # Токенизация с указанием языка
        tokens = [word for word in tokens if word.isalnum() and word not in stop_words]  # Фильтрация
        tokens = [stemmer.stem(word) for word in tokens]  # Стемминг для русского языка
        preprocessed_texts.append(tokens)

    return preprocessed_texts
preprocessed_texts = preprocessing(text)

In [ ]:
preprocessing(text)

[['хран', 'дан', 'вид'],
 ['полиморфизм', 'позволя', 'использова', 'интерфейс', 'разн', 'тип'],
 ['рекурс', 'эт', 'выз', 'функц', 'сам']]

Векторизация

In [ ]:
def make_dict(texе: list[str]) -> dict:
    preprocessed_texts = preprocessing(text)
    vocab = sorted(set(word for text in preprocessed_texts for word in text))
    return {i: word for i, word in enumerate(vocab)}

In [ ]:
make_dict(text)

{0: 'вид',
 1: 'выз',
 2: 'дан',
 3: 'интерфейс',
 4: 'использова',
 5: 'позволя',
 6: 'полиморфизм',
 7: 'разн',
 8: 'рекурс',
 9: 'сам',
 10: 'тип',
 11: 'функц',
 12: 'хран',
 13: 'эт'}

Bag of Words

In [ ]:
unique_words = []
for sublist in preprocessed_texts:
    for word in sublist:
        if word not in unique_words:
            unique_words.append(word)
unique_words = sorted(unique_words)
print(unique_words)

['вид', 'выз', 'дан', 'интерфейс', 'использова', 'позволя', 'полиморфизм', 'разн', 'рекурс', 'сам', 'тип', 'функц', 'хран', 'эт']


In [ ]:
make_dict(text)

{0: 'вид',
 1: 'выз',
 2: 'дан',
 3: 'интерфейс',
 4: 'использова',
 5: 'позволя',
 6: 'полиморфизм',
 7: 'разн',
 8: 'рекурс',
 9: 'сам',
 10: 'тип',
 11: 'функц',
 12: 'хран',
 13: 'эт'}

Создаем матрицу мешка слов

In [ ]:
def bag_of_words(tokenized_texts: list[list[str]], vocabulary: list[str]) -> np.ndarray:

    matrix = []
    for sublist in tokenized_texts:
        vector = [sublist.count(word) for word in vocabulary]
        matrix.append(vector)
    return np.array(matrix)

bow_matrix = bag_of_words(preprocessed_texts, unique_words)
print(bow_matrix)

[[1 0 1 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 1 1 1 1 1 0 0 1 0 0 0]
 [0 1 0 0 0 0 0 0 1 1 0 1 0 1]]


TF-IDF

Рассчитываем TF-IDF матрицу

In [ ]:
def manual_tfidf(bow_matrix):

    tf = bow_matrix / (bow_matrix.sum(axis=1, keepdims=True))

    idf = np.array([log(len(bow_matrix) / (1 + np.count_nonzero(bow_matrix[:, i]))) for i in range(bow_matrix.shape[1])])

    tfidf = tf * idf
    return tfidf
tfidf_matrix = manual_tfidf(bow_matrix)
print(np.round(tfidf_matrix, 8)) # Можно сделать меньше для лучшей читаемости

[[0.13515504 0.         0.13515504 0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.13515504 0.        ]
 [0.         0.         0.         0.06757752 0.06757752 0.06757752
  0.06757752 0.06757752 0.         0.         0.06757752 0.
  0.         0.        ]
 [0.         0.08109302 0.         0.         0.         0.
  0.         0.         0.08109302 0.08109302 0.         0.08109302
  0.         0.08109302]]
